# 1. Setup

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import json
import logging
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import regex as re
import typer
from dateutils import relativedelta
from unidecode import unidecode

from analysis.config import *
from analysis.preprocessing import map_progress, tokenizer_lemma
from analysis.utils import *
from src.custom_logger import get_logger

# Removendo aviso de debug
os.environ["PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT"] = "3000"
# Removendo limitação de print de dfs
# pd.set_option("display.max_rows", 500)
# pd.set_option("display.max_columns", 500)
# pd.set_option("display.width", 1000)

# Caminho da pasta contendo os csvs
data_path = "data/2023/01/19/"
places_file = "data/places.csv"
input_file = "input/hotels_23_01_19.csv"
reports_folder = Path("./reports")
Path(reports_folder).mkdir(exist_ok=True)
data_folder = Path("./data/prep")
Path(data_folder).mkdir(exist_ok=True)


In [3]:
from tqdm import tqdm
tqdm.pandas()

In [4]:
from itertools import chain

join_str = lambda x: "\n".join(list(x)) if x is not None else ""
join_list = lambda x: list(chain.from_iterable(x))

# 2. Load

In [5]:
df_prep = pd.read_parquet("data/prep/df_text_prep_2023-06-22_20-52-11_175568.pq")
df_hotels = pd.read_parquet("data/prep/df_hotels_2023-06-22_20-52-11_175568.pq")

In [6]:
df_points = pd.read_parquet("data/processed/df_points_2023-07-10_11-13-12_148931.pq")
df_sentiment = pd.read_parquet("data/processed/df_sentiments_2023-07-10_09-48-22_879161.pq")
df_topics = pd.read_parquet("data/processed/df_topics_2023-07-09_23-48-54_286392.pq")

In [23]:
df_merged = pd.concat([
    df_points, 
    df_topics.filter(regex="^lda"), 
    df_sentiment.filter(regex="^vader|^rating|^like|^text$")
], axis=1)

In [80]:
df_merged = df_merged.drop(columns="lda_topics")

# SumOpinions - With Kmeans

In [104]:
# import kmeans
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

summary_ids = {}

for hotel in df_merged.name.sort_values().unique():
    df_hotel = df_merged[df_merged.name == hotel].copy()
    df_hotel = df_hotel.drop(columns=["name", "text","lda_first_topic_name"])
    df_hotel = df_hotel.set_index("review_id", verify_integrity=True)
    df_hotel = df_hotel.dropna()

    kmeans = KMeans(n_clusters=5, random_state=0).fit(df_hotel)

    df_hotel["cluster"] = kmeans.labels_
    df_hotel["centroid"] = False

    for cluster in df_hotel.cluster.unique():
        df_cluster = df_hotel[df_hotel.cluster == cluster]
        df_cluster = df_cluster.drop(columns=["cluster","centroid"])
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_[cluster].reshape(1,14), df_cluster)
        closest_idx = df_cluster.iloc[closest[0]].name
        df_hotel["centroid"].loc[closest_idx] = True

    summary_ids[hotel] = df_hotel[df_hotel.centroid].index.tolist()


C:\Users\Bruno\AppData\Local\Temp\ipykernel_18368\2639214847.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotel["centroid"].loc[closest_idx] = True
C:\Users\Bruno\AppData\Local\Temp\ipykernel_18368\2639214847.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotel["centroid"].loc[closest_idx] = True
C:\Users\Bruno\AppData\Local\Temp\ipykernel_18368\2639214847.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hote

In [109]:
summary_kmeans_texts = {}

for hotel in summary_ids:
    t = df_merged[df_merged.review_id.isin(summary_ids[hotel])].text.tolist()
    summary_kmeans_texts[hotel] = "\n".join(t)

In [112]:
print(summary_kmeans_texts["Atlântico Center"])

Hotel não condiz com o anúncio. Atendimento péssimo, os quartos estavam sujos, o café da manhã piorado, só pão queijo e presunto café com leite e água. Enfim não gostei
Hotel muito bom pra quem desejo ficar no centro do RJ. Me hospedei lá várias vezes. O café é bem melhor do que o de alguns hoteis ao redor e que são mais caros. O atendimento é razoável (lembre que aqui é RJ, nao espere muita coisa). Os quartos são bons para o preço que eles cobram, só dava pra melhorar o ar condicionado (não fica frio, mas também não fica quente). No geral, merece 5 estrelas!
Tranquilo, muito confortável com exceção da internet que claramente não existe.
Hotel péssimo, roupas de cama encardidas, toalhas parecem panos de chão, não são perfumadas com aspecto de limpeza, móveis velhos ar condicionado antigo, banheiro sem aspecto de limpeza, acho q só tiram o lixo da pessoa anterior, não desinfetam para o próximo hóspede ,precisa urgente de reforma estrutural mas, mais que estrutura o conforto com os itens

# Summarization - Arbitrary Score

In [108]:
summary_score_texts = {}

for hotel in df_merged.name.unique():
    df_hotel = df_merged[df_merged.name == hotel]
    summary = []
    # Points
    summary.append(df_hotel.sort_values("text_points", ascending=False).iloc[0].review_id)
    # LDA points
    for i in range(6):
        df_ = df_hotel[df_hotel.lda_first_topic == i]
        summary.append(df_.sort_values(["text_points"], ascending=False).iloc[0].review_id)
    # Vader
    summary.append(df_hotel.sort_values([f"vader_compound", "text_points"], ascending=False).iloc[0].review_id)
    summary.append(df_hotel.sort_values([f"vader_compound", "text_points"], ascending=[True, False]).iloc[0].review_id)
    summary_list = df_hotel.loc[df_hotel.review_id.isin(summary)].text
    summary_text = "\n".join(summary_list)
    print(f"hotel:{hotel}\n{summary_text}")
    
    summary_score_texts[hotel] = summary_text


hotel:Atlântico Center
Como sempre era de costume estava acostumada a me hospedar no Center Hotel todo Sábado e Domingo. Porém hoje dia 14/09 houve um incidente extremamente delicado e chato. A reserva que meu namorado havia feito no dia 01/09 e que estava confirmada não foi realizada. Chegamos no Hotel as 15:30 ( sendo que o check é as 14:00 ) o funcionário Felipe nos disse que NÃO HAVIA QUARTO DISPONÍVEL. O mais engracado vem agora: assim que chegamos tinha um casal trocando de quarto, logo, tinha um quarto vazio. Certo?! Lamentável. Infelizmente lá fomos eu e meu namorado cheio de malas e peso procurar um outro Hotel. Eles recomendaram um Hotel alí perto porém era R$40,00 reais mais caro, como não tivemos opção, ficamos nesse hotel mesmo, por sinal, mil vezes melhor. Hotel Villa Reggia. Infelizmente acionei meu advogado porque o constrangimento que passei não desejo a mais ninguém e as medidas cabíveis serão tomadas. NÃO RECOMENDO O CENTER HOTEL POIS ELE NÃO É CONFIAVEL.
Quando ver 

In [113]:
print(summary_score_texts["Atlântico Center"])

Como sempre era de costume estava acostumada a me hospedar no Center Hotel todo Sábado e Domingo. Porém hoje dia 14/09 houve um incidente extremamente delicado e chato. A reserva que meu namorado havia feito no dia 01/09 e que estava confirmada não foi realizada. Chegamos no Hotel as 15:30 ( sendo que o check é as 14:00 ) o funcionário Felipe nos disse que NÃO HAVIA QUARTO DISPONÍVEL. O mais engracado vem agora: assim que chegamos tinha um casal trocando de quarto, logo, tinha um quarto vazio. Certo?! Lamentável. Infelizmente lá fomos eu e meu namorado cheio de malas e peso procurar um outro Hotel. Eles recomendaram um Hotel alí perto porém era R$40,00 reais mais caro, como não tivemos opção, ficamos nesse hotel mesmo, por sinal, mil vezes melhor. Hotel Villa Reggia. Infelizmente acionei meu advogado porque o constrangimento que passei não desejo a mais ninguém e as medidas cabíveis serão tomadas. NÃO RECOMENDO O CENTER HOTEL POIS ELE NÃO É CONFIAVEL.
Quando ver ou ouvir falar desse ho

# Evaluation

In [114]:
full_texts = {}

for hotel in df_prep.name.unique():
    full_texts[hotel] = df_hotels.loc[df_hotels.name == hotel].text_sum.iloc[0]

In [115]:
full_texts["Atlântico Center"]

'Bom preco ,Ótima Localização,atendimento bom ,quarto espaçoso com ar condicionado  .  Banheiro com toalhas ,chuveiro quebrado só o cano ,café da manhã com poucas opções .  Se arrumar o chuveiro e melhorar o café,dá pra cobrar um pouco mais caro a diária .  \n Eu ,nunca vi um quarto de hotel tão sujo na minha vida .  Lençol de cama sujo , travesseiro horrível, sem toalha de rosto, além disso, fui cobrada duas vezes no crédito, não existe serviço de atendimento ao cliente .  Entrei em contato, ninguém me deu bola .  Não recomendo para ninguém .  Sai fora, lugar imundo .  \n Hotel de baixíssima qualidade no atendimento, nas acomodações, elevadores quebrados, péssimas acomodações, roupa de cama imunda .  se vc quiser passar raiva pode ir que vai passar bastante .  \n Péssimo atendimento, muito clássico do Rio de Janeiro, não liparam o quarto em nenhum momento durante a estadia, nem se quer repunha papel higiene e o box do banheiro não passou meu amigo que tem um porte grande .  Não recome

In [116]:
summary_chat_gpt = """
Localização: O hotel está bem localizado no centro do Rio de Janeiro, próximo a pontos turísticos, museus e estações de transporte público, como o VLT. Alguns avaliadores destacaram a boa mobilidade oferecida pela localização.
Condições do hotel: Vários avaliadores relataram problemas com as condições do hotel. Muitos mencionaram que o hotel é antigo e precisa de reformas. Alguns quartos apresentavam cheiro de mofo, camas antigas e colchões desconfortáveis. Também foram mencionados problemas com a limpeza, como roupas de cama sujas e banheiros malcheirosos.
Atendimento: Houve opiniões divergentes sobre o atendimento. Alguns avaliadores elogiaram a prestatividade e cordialidade da equipe, enquanto outros mencionaram atendentes mal educados e com descaso. Alguns mencionaram problemas de comunicação devido à falta de proficiência em português.
Café da manhã: O café da manhã recebeu opiniões mistas. Alguns avaliadores consideraram satisfatório, com variedade de opções, frutas frescas e bom serviço. No entanto, outros mencionaram que o café da manhã era básico e com poucas opções.
Wi-Fi e comodidades: Vários avaliadores relataram problemas com o Wi-Fi, mencionando que a conexão era fraca ou inexistente nos quartos. Além disso, alguns mencionaram a falta de comodidades, como piscina e televisão a cabo com poucos canais.
Preço: Alguns avaliadores consideraram o hotel com um bom custo-benefício, enquanto outros acharam caro pelo que era oferecido. Houve comentários sobre taxas adicionais cobradas no momento do check-in, o que gerou insatisfação.
Barulho: Muitos avaliadores mencionaram problemas com o barulho. O tráfego intenso na rua, as obras nos prédios vizinhos e o ruído dos elevadores foram citados como causadores de incômodo durante a estadia.
Segurança: Alguns avaliadores mencionaram questões de segurança, como falta de segurança nos pertences e entrada de pessoas nos quartos sem autorização.
"""

In [117]:
summary_google = """
Quem não quer gastar muito e pretende ficar no quarto, até vale a pena.
Lá eles tem um café da manhã super simples com: café com leite, apresentudado, queijo, pão e banana.
Não achei muito higiênico na parte dos lençóis, manchas amarelas e aparentemente velhos. Camas e travesseiro confortáveis, mas precisam de lençóis brancos e novos.
No final de semana a região não vale muito a pena, pois os comércios locais são bem escassos.
No geral não voltaria para o Hotel, tem opções melhores.

A localização é ótima, pertinho de muita coisa legal, fácil acesso. Recepção cordiais.
Limpeza precária lençóis e toalhas encardidos, colcha das camas sujas, ar condicionado ruim, telefone do quarto desligado, wi-fi só na recepção, TV sem opções de canais.

Antes de fazer uma reserva,  veja as avaliações,  eu na empolgação do momento não vi, e paguei pela falha,  este hotel meia estrela já é muito,  pra fica ruim falta muito, quartos horrível,  velho, cheirando a mofo, Internet esqueci não tem, higiene nos quartos não fazem e quando eu solicitei depois de três dias disseram que não poderia ser feito,  aleguei que iria fazer avaliação,  e o funcionário me orientou a fazer mesmo negativa, o tempo que fiquei as toalhas não foram trocadas mesmo quando solicitei, tv e ar não funciona direito antigos não recomendo,  a menos q vc queira paga pecados e passa muita raiva,  o café então,  não tem um suco uma fruta pra variar,  só pão, apresuntado e queijo,  café e leite só raiva mesmo, três estrelas,  tem que se avaliado novamente,  eu das três estrelas tirava quatro

Paguei 40 conto na diária, ja era de se imaginar que não era o melhor hotel da cidade. Mas vale as 5 estrelas porque é muito bem localizado e da pra ir andando até o aeroporto. Ou seja, pra quem quer hotel barato e bem localizado esse é bom!
"""

## Rouge

In [129]:
import rouge_score
from rouge_score import rouge_scorer

scorer = rouge_score.rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3'], use_stemmer=True)

def get_rouge_score(text, summary, scorer=scorer):
    score = scorer.score(text, summary)
    for metric, results in score.items():
        print(f"{metric}: precision={results.precision:.4f}; recall={results.recall:.4f}; fmeasure={results.fmeasure:.4f}")

print("summary_kmeans")
get_rouge_score(full_texts["Atlântico Center"], summary_kmeans_texts["Atlântico Center"])


summary_kmeans
rouge1: precision=1.0000; recall=0.0099; fmeasure=0.0196
rouge2: precision=0.9855; recall=0.0097; fmeasure=0.0192
rouge3: precision=0.9709; recall=0.0095; fmeasure=0.0189


In [130]:
print("summary_score")
get_rouge_score(full_texts["Atlântico Center"], summary_score_texts["Atlântico Center"])

summary_score
rouge1: precision=1.0000; recall=0.0390; fmeasure=0.0751
rouge2: precision=0.9954; recall=0.0388; fmeasure=0.0747
rouge3: precision=0.9899; recall=0.0385; fmeasure=0.0742


In [131]:
print("summary_chat_gpt")
get_rouge_score(full_texts["Atlântico Center"], summary_chat_gpt)

summary_chat_gpt
rouge1: precision=0.8847; recall=0.0101; fmeasure=0.0201
rouge2: precision=0.5250; recall=0.0060; fmeasure=0.0119
rouge3: precision=0.1975; recall=0.0022; fmeasure=0.0044


In [134]:
print(" summary_google")
get_rouge_score(full_texts["Atlântico Center"], summary_google)

 summary_google
rouge1: precision=0.9657; recall=0.0121; fmeasure=0.0238
rouge2: precision=0.6619; recall=0.0082; fmeasure=0.0163
rouge3: precision=0.4454; recall=0.0055; fmeasure=0.0109


## Bleu

In [73]:
import evaluate

# Define the candidate predictions and reference sentences
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi", "hello there !"],["foo bar foobar"]]

# Load the BLEU evaluation metric
bleu = evaluate.load("bleu")

# Compute the BLEU score
results = bleu.compute(predictions=predictions, references=references)

# Print the results
print(results)

{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.1666666666666667, 'translation_length': 7, 'reference_length': 6}


## Legibilidade

In [ ]:
# https://legibilidade.com/